In [1]:
from flask import Flask, request, jsonify, render_template
import random
import tensorflow
from tensorflow.keras.models import load_model
import numpy as np
import librosa
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import os

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
app = Flask(__name__, template_folder='template', static_folder='static')
model=load_model(r'C:\Users\user\Documents\Projects_Docs\SER_Emergency\Emotion_Emergency.h5')

: 

In [ ]:
@app.route('/')
def home():
    return render_template('index.html')

: 

In [ ]:
@app.route('/predict_',methods=['POST'])
def predict_():

    # get file from POST request and save it
    audio_file = request.files
    file = audio_file.get('file')
    print(file)

    with open(os.path.abspath(f'backend/audios/{file}'), 'wb') as f:
        f.write(file.content)

    response = jsonify("File received and saved!")
    response.headers.add('Access-Control-Allow-Origin', '*')

    # return response
    # audio_file = request.files["file"]
    file_name = str(random.randint(0, 100000))
    audio_file.save(file_name)

    # Features = pd.read_csv(r'C:\Users\user\Documents\Projects_Docs\SER_Emergency\features_Emergency.csv')
    # Y = Features['labels'].values
    # encoder = OneHotEncoder()
    # Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
    
    # NOISE
    def noise(data):
        noise_amp = 0.035*np.random.uniform()*np.amax(data)
        data = data + noise_amp*np.random.normal(size=data.shape[0])
        return data
    # # STRETCH
    # def stretch(data, rate=0.8):
    #     return librosa.effects.time_stretch(data, rate)
    # # PITCH
    # def pitch(data, sampling_rate, pitch_factor=0.7):
    #     return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

    def feat_ext(data):
        mfcc = np.mean(librosa.feature.mfcc(y=data, sr=22050).T, axis=0)
        return mfcc

    def get_feat(path):
        data, sample_rate = librosa.load(path, duration=5, offset=0.6)
        # normal data
        res1 = feat_ext(data)
        result = np.array(res1)
        #data with noise
        noise_data = noise(data)
        res2 = feat_ext(noise_data)
        result = np.vstack((result, res2))
        return result

    feature = get_feat(file_name)
    test =np.expand_dims(feature, axis=2)
    livepreds = model.predict(test)
    livepredictions = (encoder.inverse_transform((livepreds)))


    return render_template('index.html', prediction_text='OUTPUT IS {}'.format(livepredictions[0],audio_file))


: 

In [ ]:
# @app.route('/results',methods=['POST'])
# def results():
#     data = request.get_json(force=True)
#     prediction = model.predict(data.values())
    
#     return jsonify(prediction)

: 

In [ ]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

: 